### Import Modules

In [1]:
import os
import sys
import json
import subprocess

import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from visualizing_chromatin_HP1_PRC1 import generate_PDB

### Load PyMol

In [2]:
#import __main__
#__main__.pymol_argv = [ 'pymol', '-qc'] # Quiet and no GUI
import pymol

### Identify Output Directories

In [3]:
### UPDATE THESE VALUES
out_path = "../../output"
polymer_prefix = "Chr-1"
sim_range_inclusive = (45, 84)
confine_radius = 452.5731490338889
###

all_content = os.listdir(out_path)
all_content_paths = [f"{out_path}/{item}" for item in all_content]
sim_paths = [path for path in all_content_paths if os.path.isdir(path)]
sim_dict = {int(path.split("_")[-1]): path for path in sim_paths}

### Filter Output Directories

In [4]:
sims = {key: sim_dict[key] for key in sim_dict.keys() if key <= sim_range_inclusive[1] and key >= sim_range_inclusive[0]}

### Identify Latest Configuration in Each Sim

In [5]:
latest_files = {}
for sim in sims.keys():
    files = os.listdir(sims[sim])
    files = [file for file in files if file.endswith(".csv") and file.startswith(polymer_prefix)]
    file_inds = [int(file.split(".")[0].split("-")[-1]) for file in files]
    latest_files[sim] = f"{sims[sim]}/{files[np.argmax(file_inds)]}"

### Characterize Interactions

In [6]:
binders_paths = {key: f"{sims[key]}/binders" for key in sims.keys()}

In [7]:
self_interact = {}
cross_interact = {}
chemical_potential = {}

for sim in binders_paths.keys():
    df_binders = pd.read_csv(binders_paths[sim], index_col="name")

    if df_binders.loc["HP1", "interaction_energy"] != df_binders.loc["PRC1", "interaction_energy"]:
        raise ValueError("Non-uniform interaction energies")
    if df_binders.loc["HP1", "chemical_potential"] != df_binders.loc["PRC1", "chemical_potential"]:
        raise ValueError("Non-uniform interaction energies")
        
    self_interact[sim] = df_binders.loc["HP1", "interaction_energy"]
    cross_interact[sim] = json.loads(df_binders.loc["HP1", "cross_talk_interaction_energy"].replace("'", "\""))["PRC1"]
    chemical_potential[sim] = df_binders.loc["HP1", "chemical_potential"]

all_data = {}
for key in self_interact.keys():
    
    str_self = str(self_interact[key]).replace("-", "m").replace(".", "d")
    str_cross = str(cross_interact[key]).replace("-", "m").replace(".", "d")
    str_cp = str(chemical_potential[key]).replace("-", "m").replace(".", "d")
    pdb_suffix = f"_self_{str_self}_cross_{str_cross}_cp_{str_cp}.pdb"
    pdb_save_suffix = f"_self_{str_self}_cross_{str_cross}_cp_{str_cp}.png"

    all_data[f"sim_{key}"] = [
        sims[key],
        latest_files[key],
        self_interact[key],
        cross_interact[key],
        chemical_potential[key],
        latest_files[key].replace(".csv", pdb_suffix),
        latest_files[key].replace(".csv", pdb_save_suffix)
    ]

parameters = pd.DataFrame.from_dict(
    all_data,
    orient='index',
    columns=[
        "output_dir",
        "latest_snapshot",
        "self_interaction",
        "cross_interaction",
        "chemical_potential",
        "pdb_path",
        "pdb_save_path"
    ]
).sort_index()

parameters.head()

,output_dir,latest_snapshot,self_interaction,cross_interaction,chemical_potential,pdb_path,pdb_save_path
sim_45,../../output/sim_45,../../output/sim_45/Chr-1-86.csv,-4.0,0.0,-1.2,../../output/sim_45/Chr-1-86_self_m4d0_cross_0...,../../output/sim_45/Chr-1-86_self_m4d0_cross_0...
sim_46,../../output/sim_46,../../output/sim_46/Chr-1-89.csv,-4.0,2.0,-1.2,../../output/sim_46/Chr-1-89_self_m4d0_cross_2...,../../output/sim_46/Chr-1-89_self_m4d0_cross_2...
sim_47,../../output/sim_47,../../output/sim_47/Chr-1-84.csv,-4.0,1.0,-0.4,../../output/sim_47/Chr-1-84_self_m4d0_cross_1...,../../output/sim_47/Chr-1-84_self_m4d0_cross_1...
sim_48,../../output/sim_48,../../output/sim_48/Chr-1-88.csv,-4.0,-1.0,-1.6,../../output/sim_48/Chr-1-88_self_m4d0_cross_m...,../../output/sim_48/Chr-1-88_self_m4d0_cross_m...
sim_49,../../output/sim_49,../../output/sim_49/Chr-1-89.csv,-4.0,-1.0,-0.4,../../output/sim_49/Chr-1-89_self_m4d0_cross_m...,../../output/sim_49/Chr-1-89_self_m4d0_cross_m...


In [8]:
parameters_dict = parameters.to_dict("records")

### Save PDB Files

In [9]:
# Make sure `visualizing_chromatin_HP1_PRC1.py` is in the same directory as this notebook!
for i in range(len(parameters_dict)):
    generate_PDB(
        f"{parameters_dict[i]['latest_snapshot']}",
        f"{parameters_dict[i]['pdb_path']}",
        f"{confine_radius}"
    )

### Render Configurations in PyMol

In [ ]:
for i in range(len(parameters_dict)):
    pymol.cmd.reinitialize()
    pymol.cmd.load(f"{parameters_dict[i]['pdb_path']}")
    pymol.cmd.show_as("spheres", "all")
    pymol.cmd.png(f"{parameters_dict[i]['pdb_save_path']}")
    pymol.cmd.delete("all")
pymol.cmd.quit()

 PyMOL not running, entering library mode (experimental)


### Create a Grid of Configurations

In [ ]:
rows = ["m0d4", "m0d8", "m1d2", "m1d6"]
columns = ["m2d0", "m1d0", "m0d5", "m0d25", "0d0", "0d25", "0d5", "1d0", "2d0"]

column_headers = [col.replace("m", "-").replace("d", ".") for col in columns]
row_headers = [row.replace("m", "-").replace("d", ".") for row in rows]


def row_cp_col_cross(file_path, rows, columns):
    """Get the row and column position of a configuration in an array of chemical potentials and cross-interactions.

    Each row corresponds to a common chemical potential and each column corresponds to a common cross-interaction.
    """
    pdb_file = file_path.split(".")[-2].split("/")[-1]
    meta_data = pdb_file.split("_")[1:]
    
    self = meta_data[meta_data.index("self")+1]
    cross = meta_data[meta_data.index("cross")+1]
    cp = meta_data[meta_data.index("cp")+1]
    
    return (rows.index(cp), columns.index(cross))


fig = plt.figure(figsize=(64,48))
grid = ImageGrid(
    fig, 111,                  
    nrows_ncols=(len(rows), len(columns)),
    axes_pad=0.1,
    share_all=True
)   

for i in range(len(parameters_dict)):
    image_file = f"{parameters_dict[i]['pdb_save_path']}"
    img = cv.imread(image_file)
    try:
        ax = grid[i]
        ax.imshow(img)
    except:
        print("Error loading file: " + image_file)

plt.savefig("../../output/Configuration_Array.png", dpi=300)
    